In [1]:
from typing import Set
import re

In [2]:
def tokenize(text: str) -> Set[str]:
    text = text.lower()                         # Convert to lowercase,
    all_words = re.findall("[a-z0-9']+", text)  # extract the words, and
    return set(all_words)                       # remove duplicates.

In [3]:
assert tokenize("Data Science is science") == {"data", "science", "is"}

In [4]:
from typing import NamedTuple

In [5]:
class Message(NamedTuple):
    text: str
    is_spam: bool

In [6]:
from typing import List, Tuple, Dict, Iterable
import math
from collections import defaultdict

In [7]:
class NaiveBayesClassifier:
    def __init__(self, k: float = 0.5) -> None:
        self.k = k  # smoothing factor

        self.tokens: Set[str] = set()
        self.token_spam_counts: Dict[str, int] = defaultdict(int)
        self.token_ham_counts: Dict[str, int] = defaultdict(int)
        self.spam_messages = self.ham_messages = 0

    def train(self, messages: Iterable[Message]) -> None:
        for message in messages:
            # Increment message counts
            if message.is_spam:
                self.spam_messages += 1
            else:
                self.ham_messages += 1

            # Increment word counts
            for token in tokenize(message.text):
                self.tokens.add(token)
                if message.is_spam:
                    self.token_spam_counts[token] += 1
                else:
                    self.token_ham_counts[token] += 1

    def _probabilities(self, token: str) -> Tuple[float, float]:
        """returns P(token | spam) and P(token | not spam)"""
        spam = self.token_spam_counts[token]
        ham = self.token_ham_counts[token]

        p_token_spam = (spam + self.k) / (self.spam_messages + 2 * self.k)
        p_token_ham = (ham + self.k) / (self.ham_messages + 2 * self.k)

        return p_token_spam, p_token_ham

    def predict(self, text: str) -> float:
        text_tokens = tokenize(text)
        log_prob_if_spam = log_prob_if_ham = 0.0

        # Iterate through each word in our vocabulary.
        for token in self.tokens:
            prob_if_spam, prob_if_ham = self._probabilities(token)

            # If *token* appears in the message,
            # add the log probability of seeing it;
            if token in text_tokens:
                log_prob_if_spam += math.log(prob_if_spam)
                log_prob_if_ham += math.log(prob_if_ham)

            # otherwise add the log probability of _not_ seeing it
            # which is log(1 - probability of seeing it)
            else:
                log_prob_if_spam += math.log(1.0 - prob_if_spam)
                log_prob_if_ham += math.log(1.0 - prob_if_ham)

        prob_if_spam = math.exp(log_prob_if_spam)
        prob_if_ham = math.exp(log_prob_if_ham)
        return prob_if_spam / (prob_if_spam + prob_if_ham)

In [8]:
messages = [Message("spam rules", is_spam=True),
            Message("ham rules", is_spam=False),
            Message("hello ham", is_spam=False)]

model = NaiveBayesClassifier(k=0.5)
model.train(messages)

In [9]:
assert model.tokens == {"spam", "ham", "rules", "hello"}

In [10]:
assert model.spam_messages == 1

In [11]:
assert model.ham_messages == 2

In [12]:
assert model.token_spam_counts == {"spam": 1, "rules": 1}

In [13]:
assert model.token_ham_counts == {"ham": 2, "rules": 1, "hello": 1}

In [14]:
text = "hello spam"

In [15]:
probs_if_spam = [
    (1 + 0.5) / (1 + 2 * 0.5),      # "spam"  (present)
    1 - (0 + 0.5) / (1 + 2 * 0.5),  # "ham"   (not present)
    1 - (1 + 0.5) / (1 + 2 * 0.5),  # "rules" (not present)
    (0 + 0.5) / (1 + 2 * 0.5)       # "hello" (present)
]

In [16]:
probs_if_ham = [
    (0 + 0.5) / (2 + 2 * 0.5),      # "spam"  (present)
    1 - (2 + 0.5) / (2 + 2 * 0.5),  # "ham"   (not present)
    1 - (1 + 0.5) / (2 + 2 * 0.5),  # "rules" (not present)
    (1 + 0.5) / (2 + 2 * 0.5),      # "hello" (present)
]

In [17]:
p_if_spam = math.exp(sum(math.log(p) for p in probs_if_spam))
p_if_ham = math.exp(sum(math.log(p) for p in probs_if_ham))

In [18]:
p_if_spam, p_if_ham

(0.03515625, 0.006944444444444444)

In [19]:
model.predict(text), p_if_spam / (p_if_spam + p_if_ham)

(0.8350515463917525, 0.8350515463917525)

In [20]:
# Should be about 0.83
assert round(model.predict(text)) == round(p_if_spam / (p_if_spam + p_if_ham))

In [21]:
def drop_final_s(word):
    return re.sub("s$", "", word)

In [22]:
# def main():

In [23]:
import glob, re
    
# modify the path to wherever you've put the files
path = r'C:\Users\Ivo\Downloads\Genius\Data_Science_From_Scratch\Cap13_Naive_Bayes\spam_data/*/*'

In [24]:
data: List[Message] = []

In [25]:
# glob.glob returns every filename that matches the wildcarded path
for filename in glob.glob(path):
    is_spam = "ham" not in filename
    
    # There are some garbage characters in the emails, the errors='ignore'
    # skips them instead of raising an exception.
    with open(filename, errors='ignore') as email_file:
        for line in email_file:
            if line.startswith("Subject:"):
                subject = line.lstrip("Subject: ")
                data.append(Message(subject, is_spam))
                break  # done with this file

In [26]:
import random
import import_ipynb
from machine_learning import split_data

importing Jupyter notebook from machine_learning.ipynb


In [27]:
random.seed(0)      # just so you get the same answers as me
train_messages, test_messages = split_data(data, 0.75)

In [28]:
model = NaiveBayesClassifier()
model.train(train_messages)

In [29]:
from collections import Counter

In [30]:
predictions = [(message, model.predict(message.text))
                   for message in test_messages]
predictions[:3]

[(Message(text='NEWS.COM INVESTOR: Techs rise, but Dow falls on day of Bush speech\n', is_spam=False),
  0.0005202358965214317),
 (Message(text='Best\n', is_spam=False), 0.14028305973242164),
 (Message(text='Grow Up And Be A Man!\xa0 abm\n', is_spam=True),
  0.11482190887633734)]

In [31]:
# Assume that spam_probability > 0.5 corresponds to spam prediction
# and count the combinations of (actual is_spam, predicted is_spam)
confusion_matrix = Counter((message.is_spam, spam_probability > 0.5)
                               for message, spam_probability in predictions)
confusion_matrix

Counter({(False, False): 670,
         (True, False): 40,
         (True, True): 86,
         (False, True): 29})

In [32]:
print(confusion_matrix)

Counter({(False, False): 670, (True, True): 86, (True, False): 40, (False, True): 29})


In [33]:
def p_spam_given_token(token: str, model: NaiveBayesClassifier) -> float:
    # We probably shouldn't call private methods, but it's for a good cause.
    prob_if_spam, prob_if_ham = model._probabilities(token)
    
    return prob_if_spam / (prob_if_spam + prob_if_ham)

In [34]:
words = sorted(model.tokens, key=lambda t: p_spam_given_token(t, model))
words[:3]

['spambayes', 'users', 'razor']

In [35]:
print("spammiest_words", words[-10:])

spammiest_words ['zzzz', '95', 'attn', 'per', 'clearance', 'money', 'rates', 'sale', 'systemworks', 'adv']


In [36]:
print("hammiest_words", words[:10])

hammiest_words ['spambayes', 'users', 'razor', 'zzzzteana', 'sadev', 'apt', 'perl', 'ouch', 'spamassassin', 'bliss']


In [37]:
# if __name__ == "__main__": main()